### 参数管理
我们首先关注具有单隐藏层的多层感知机

In [44]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3529],
        [-0.3529]], grad_fn=<AddmmBackward0>)

**参数访问**

In [45]:
print(net[2].state_dict())  # state_dict()返回一个从参数名称映射到参数Tensor的字典，也是一个OrderedDict

OrderedDict([('weight', tensor([[-0.0118, -0.0188,  0.0211,  0.3092,  0.1318,  0.2139,  0.2885,  0.3302]])), ('bias', tensor([-0.3529]))])


**目标参数**

In [46]:
print(type(net[2].bias))  # Parameter是一个可以优化的参数类型
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3529], requires_grad=True)
tensor([-0.3529])


In [47]:
net[2].weight.grad == None  # 还没有计算梯度

True

**一次性访问所有参数**

In [48]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# `net[0].named_parameters()`返回一个生成器，该生成器会产生子模块（在这种情况下是第一个子模块）的参数及其名称。每个参数都是一个张量，而名称是一个字符串。
# `name`是参数的名称，`param`是参数的张量。
# `[(name, param.shape) for name, param in net[1].named_parameters()]`是一个列表推导式，它遍历`net[0]`子模块的参数，并为每个参数生成一个元组，元组包含参数的名称和形状。
# `print(*[(name, param.shape) for name, param in net[0].named_parameters()])`使用`print`函数打印了这个列表中的每个元组。`*`符号用于展开列表，使其成为`print`函数的多个参数，从而以易读的方式打印出参数名称和形状。

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [49]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.0824, -0.0910, -0.3516, -0.1090],
                      [-0.1472, -0.0046,  0.4202, -0.4714],
                      [ 0.1747,  0.1283, -0.1174, -0.3303],
                      [ 0.2627,  0.2043, -0.3790, -0.0744],
                      [ 0.2203,  0.2846, -0.1270,  0.2337],
                      [ 0.0009,  0.3345, -0.1673, -0.3149],
                      [ 0.4409,  0.4402, -0.2779,  0.0494],
                      [-0.3236, -0.2468,  0.2035,  0.3927]])),
             ('0.bias',
              tensor([-0.2696, -0.2534, -0.2264, -0.2319, -0.4551, -0.1495, -0.2868, -0.3252])),
             ('2.weight',
              tensor([[-0.0118, -0.0188,  0.0211,  0.3092,  0.1318,  0.2139,  0.2885,  0.3302]])),
             ('2.bias', tensor([-0.3529]))])

In [50]:
net.state_dict()["2.bias"].data

tensor([-0.3529])

**从嵌套块收集参数**

In [51]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f"block{i}", block1())  # add_module是nn.Sequential类的方法，可以起名字
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4243],
        [-0.4243]], grad_fn=<AddmmBackward0>)

**我们已经设计了网络，让我们看看它是如何组织的**

In [52]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


**指定block2打印**

In [53]:
print(rgnet[0][2])

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
)


In [54]:
# 指定block2的第0层的权重参数
rgnet[0][2][0].weight.data

tensor([[-0.2688,  0.1746, -0.4798,  0.1324],
        [ 0.3981, -0.4926,  0.3337,  0.3580],
        [-0.2411,  0.4373, -0.4926,  0.2843],
        [-0.3482,  0.0212,  0.2717,  0.0149],
        [-0.0389, -0.3990, -0.0012,  0.3238],
        [ 0.1029, -0.4920, -0.4060,  0.1201],
        [-0.1901, -0.3325,  0.0095, -0.2725],
        [-0.3260, -0.2717, -0.4422, -0.3238]])

**内置初始化**

In [55]:
def init_normal(m):  # m is a layer
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


net.apply(init_normal)  # 对所有层使用init_normal函数初始化
net[0].weight.data[0], net[0].bias.data[0]  # 查看第一层的第一行的参数

(tensor([ 0.0068,  0.0050, -0.0151,  0.0032]), tensor(0.))

In [56]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

**对某些块应用不同的初始化方法**

In [57]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)


net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5401,  0.3312, -0.6944, -0.2573])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


**自定义初始化**

In [58]:
net, [(name, param.shape) for name, param in net[2].named_parameters()][0]

(Sequential(
   (0): Linear(in_features=4, out_features=8, bias=True)
   (1): ReLU()
   (2): Linear(in_features=8, out_features=1, bias=True)
 ),
 ('weight', torch.Size([1, 8])))

In [59]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)

        # 使用逐元素的比较运算 `m.weight.data.abs() >= 5` 来创建一个布尔张量，表示权重绝对值大于等于 5 的位置
        # 通过执行 `m.weight.data *= m.weight.data.abs() >= 5`，将权重中绝对值大于等于 5 的元素置零，从而达到一个稀疏化的效果
        m.weight.data *= m.weight.data.abs() >= 5


net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[5.3864, 0.0000, 5.1624, -0.0000],
        [6.4122, 6.8998, -0.0000, 7.1012]], grad_fn=<SliceBackward0>)

In [62]:
net[0].weight.data[:] += 1  # 原地改变
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  3.0000,  8.1624,  3.0000])

-----------------------

### **参数绑定**

In [69]:
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
